### Markov Chain
- 확률을 기반으로 함
- 과거의 상태를 무시하고, 현재의 상태만을 기반으로 다음 상태 선택
- 문자 단위의 Ngram과 달리 단어 단위 처리
- 실질적인 의미 연관성 무시
- 형태소 분석 -> 사전을 기반으로 문장 만들기

In [2]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
import urllib.request
import os, re,json,random

##### Markov Chain dictionary

In [16]:
def make_dic(words) :
    tmp = ['@']
    dic = {}
    
    for word in words :
        tmp.append(word)
        if len(tmp) <3 : continue
        if len(tmp) >3 : tmp = tmp[1:]
            
        set_word(dic, tmp)
        
        if word == "." :
            tmp = ["@"]
            continue
            
    return dic

##### Add to dictionary

In [17]:
def set_word(dic, s3) :
    w1,w2,w3 = s3
    if not w1 in dic : dic[w1] = {}
    if not w2 in dic[w1] : dic[w1][w2] = {}
    if not w3 in dic[w1][w2] : dic[w1][w2][w3] = 0
    
    dic[w1][w2][w3] +=1 

##### make sentences

In [18]:
def make_sentence (dic) :
    ret = []
    if not '@' in dic :
        return 'no dic'
    
    top = dic['@']
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    
    ret.append(w1)
    ret.append(w2)
    
    while True :
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == "." :
            break
        w1,w2 = w2,w3
        
    #띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
    
    # 네이버 맞춤법 검사기를 사용합니다.
    request = urllib.request.Request("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params)
    request.add_header('Referer', 'https://search.naver.com/earch.naver')
    data = urllib.request.urlopen(request)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data

In [19]:
def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

In [20]:
# 문장 읽어 들이기 --- (※4)
toji_file = "toji.txt"
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("C:/Users/wjssm/Desktop/[스터디] 파이썬_머신러닝/data/BEXX0003.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("body > text")
    text = body.getText()
    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    # 형태소 분석
    twitter = Okt()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))
# 문장 만들기 --- (※6)
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print("---")

HTTPError: HTTP Error 403: Forbidden